<a href="https://colab.research.google.com/github/leaguilar/AIknowthatfeel/blob/master/Workshop1_example_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Collab

#### git clone

In [49]:
#!git clone https://github.com/<your name>/<your project>.git
#pip install -r requirements.txt

#### Import libraries


In [50]:
import numpy as np
import os
import shutil
import datetime

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [52]:
from libs.workshop_libs.tensorboard import TrainValTensorBoard

### Import dataset

In [53]:
from tensorflow.keras.datasets import imdb

In [54]:
# modify the default parameters of np.load  ### See below (problems with old numpy versions)
# save np.load
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [55]:
top_words = 5000
max_review_length = 500
pad_char=0
start_char=1
oov_char=2
index_from=3
test_split=0.2

In [56]:
(x_train, y_train), (x_dev, y_dev) = imdb.load_data(seed=1,
                                                      num_words=top_words,
                                                      maxlen=max_review_length,
                                                      start_char=start_char,
                                                      oov_char=oov_char,
                                                      index_from=index_from
                                                     )

In [57]:
# restore np.load for future normal usage ### See below (problems with old numpy versions)
np.load = np_load_old

In [58]:
x_train.shape

(25000,)

In [59]:
len(x_train)

25000

In [60]:
len(x_train[0])

284

In [61]:
x_train[0][:10]

[1, 13, 28, 1039, 7, 14, 23, 1856, 13, 104]

In [62]:
x_dev[0][:10]

[1, 13, 286, 1017, 3845, 3561, 21, 13, 16, 1383]

In [63]:
max_data=10000
x_train=x_train[0:max_data]
y_train=y_train[0:max_data]
x_dev=x_dev[0:max_data]
y_dev=y_dev[0:max_data]

### Let's check the dataset

In [64]:
wordIDX = imdb.get_word_index()
wordIDX = {k:(v+index_from) for k,v in wordIDX.items()}
wordIDX["<>"] = pad_char
wordIDX["<INIT>"] = start_char
wordIDX["<?>"] = oov_char
IDXword = {value:key for key,value in wordIDX.items()}

IDXsentiment={}
IDXsentiment[0]="positive"
IDXsentiment[1]="negative"   

In [65]:
def printExample(x,y,i,IDXword,IDXsentiment):
    print("="*10)
    print(' '.join(IDXword[i] for i in x[i] ))
    print('-'*10)
    print('Sentiment: {}'.format(IDXsentiment[y[i]]))

In [66]:
printExample(x_train,y_train,0,IDXword,IDXsentiment)
printExample(x_train,y_train,1,IDXword,IDXsentiment)
printExample(x_dev,y_dev,1,IDXword,IDXsentiment)

<INIT> i have copy of this on vhs i think they the television <?> should play this every year for the next twenty years so that we don't forget what was and that we remember not to do the same mistakes again like putting some people in the director's chair where they don't belong this movie <?> is like a <?> musical for those who can't sing or act this movie is as much fun as trying to teach the <?> to drive a city bus br br john hood <?> has just got out of prison and he's headed back to the old neighborhood in serving time for an all to nice crime of <?> of course john heads back onto the old street and is <?> by kids dogs old ladies and his <?> <?> as they dance and sing all along the way br br i would recommend this if i was sentimental or if in truth someone was smoking <?> pot <?> by a doctor for <?> either way this is a poorly directed scripted acted and even produced i never thought i'd sat that satire of <?> life with the <?> although i think the redeeming part of the story th

In [67]:
x_dev_list=x_dev.tolist()

In [68]:
x_train[100] in x_dev_list

False

In [69]:
## Load tensorboard

In [70]:
%load_ext tensorboard
#%reload_ext tensorboard

The tensorboard module is not an IPython extension.


In [71]:
#!kill 4430

In [72]:
#!ps aux | grep -i tb

In [73]:
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
print(shutil.rmtree(logs_base_dir))

None


In [74]:
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

UsageError: Line magic function `%tensorboard` not found.


In [75]:
### Clean data, truncate y padding

In [76]:
def prepare_data(x_train,x_dev,max_review_length):
    x_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
    x_dev = sequence.pad_sequences(x_dev, maxlen=max_review_length)
    return x_train,x_dev

In [77]:
def create_model(top_words,max_review_length):
    embedding_vector_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())
    return model

In [78]:
def train_model(model,x_train,y_train,x_dev,y_dev):
        logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy']
                     )
        model.fit(x_train,
                  y_train,
                  batch_size=128, #64,
                  validation_data=(x_dev, y_dev),
                  nb_epoch=10,
                  callbacks=[TrainValTensorBoard(logdir, 
                                                 histogram_freq=1,
                                                 write_graph=True
                                                )]
                 )

In [79]:
x_train,x_dev = prepare_data(x_train,x_dev,max_review_length)

In [80]:
print(x_train.shape)
print(x_dev.shape)

(10000, 500)
(10000, 500)


In [81]:
model=create_model(top_words,max_review_length)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
train_model(model,x_train,y_train,x_dev,y_dev)

/home/leo/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 56s 6ms/step - loss: 0.6511 - acc: 0.6115 - val_loss: 0.5053 - val_acc: 0.7677
Epoch 2/10
10000/10000 [==============================] - 62s 6ms/step - loss: 0.3754 - acc: 0.8419 - val_loss: 0.4208 - val_acc: 0.8403
Epoch 3/10
10000/10000 [==============================] - 62s 6ms/step - loss: 0.2614 - acc: 0.8998 - val_loss: 0.4132 - val_acc: 0.8574
Epoch 4/10
10000/10000 [==============================] - 57s 6ms/step - loss: 0.2001 - acc: 0.9286 - val_loss: 0.4775 - val_acc: 0.8287
Epoch 5/10
10000/10000 [==============================] - 56s 6ms/step - loss: 0.1718 - acc: 0.9373 - val_loss: 0.7393 - val_acc: 0.7909
Epoch 6/10
10000/10000 [==============================] - 55s 6ms/step - loss: 0.2427 - acc: 0.9054 - val_loss: 0.3898 - val_acc: 0.8460
Epoch 7/10
10000/10000 [==============================] - 57s 6ms/step - loss: 0.1325 - acc: 0.9534 - val_loss: 0.4058 - val_acc

### Let's evaluate the model

In [39]:
scores = model.evaluate(x_dev, y_dev, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.83%


### Save the model

In [40]:
outpath='models/'
os.makedirs(outpath, exist_ok=True)

In [41]:
model.save(outpath+"model_other.h5")

In [42]:
# serialize model to JSON
model_json = model.to_json()
with open(outpath+"model_other.json", "w") as json_file:
    json_file.write(model_json)

### One hot encoder

In [44]:
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.text import one_hot

In [45]:
texto=["This is a phrase","This is a longer phrase", "another phrase"]

In [46]:
vocab_size = 50
vectorized_words = [one_hot(d, vocab_size) for d in texto]
print(vectorized_words)

[[29, 6, 17, 49], [29, 6, 17, 39, 49], [7, 49]]


### Problems with some numpy versions and keras

In [47]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
#(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [48]:
# restore np.load for future normal usage
np.load = np_load_old

In [ ]:
### Another solution

In [ ]:
#!pip uninstall numpy

In [ ]:
#!pip install numpy==1.16.1